2019 Census Data for Selected Variables - Baltimore City

In [1]:
# STILL NEED TO CALCULATE THE FOLLOWING STATS:
# DAYTIME POPULATION, HOME OWNERSHIP RATE, HH DENSITY, HH GROWTH, INCOME GROWTH, POP DENSITY, POP GROWTH

In [2]:
#pip install us

In [3]:
# pip install censusgeocode
# pip install censusdata

In [4]:
# pip install git+git://github.com/geopandas/geopandas.git

In [5]:
# From https://www.census.gov/programs-surveys/acs/guidance/comparing-acs-data.html:
# "Due to the impact of the COVID-19 pandemic, the Census Bureau changed the 2020 ACS release. 
# Instead of providing the standard 1-year data products, the Census Bureau released experimental estimates from the 1-year data. 
# Data users should not compare 2020 ACS 1-year experimental estimates with any other data.""

In [6]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
# import censusdata 
import censusgeocode as cg
#import gmaps

# Census & gmaps API Keys
from config import api_key
c = Census(api_key, year=2019)

# Configure gmaps
#gmaps.configure(api_key=gkey)

In [7]:
# testing search function
#sample = censusdata.search('acs1', 2015, 'concept', 'employment')
#print(sample)


In [8]:
#censusdata.printtable(censusdata.censustable('acs1', 2019, 'B08604'))

In [9]:
# Run Census Search to retrieve data on Baltimore City, MD (all census tracts in Baltimore City)
# ***See https://api.census.gov/data/2019/acs/acs5/groups.html  for list of variables and groups for the ACS 5-year estimates***
# ex. "B23025_005E" is "unemployment count"
census_data = c.acs5.state_county_tract(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E",
                          "B23025_004E",                                                            
                          "B15003_017E",
                          "B15003_022E",                
                          "B02001_002E",
                          "B02001_003E",
                          "B02001_008E",
                          "B03001_003E",
                          "B25008_002E",
                          "B28007_001E",
                          "B25003_002E",               
                          "B25003_003E"),               
                          state_fips = "24",
                          county_fips = "510",
                          tract = "*") 

# convert to dataframe
census_pd = pd.DataFrame(census_data)
census_pd.head()
                         

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_003E,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,1741.0,0.0,0.0,306.0,1218.0,119.0,622.0,24,510,260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,169.0,99.0,88.0,1030.0,1808.0,539.0,443.0,24,510,230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,4254.0,0.0,94.0,3674.0,3812.0,1384.0,323.0,24,510,260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,626.0,29.0,66.0,164.0,716.0,88.0,368.0,24,510,260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,310.0,10.0,970.0,751.0,1875.0,415.0,580.0,24,510,260800


In [10]:
# Add in daytime population
# See https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "commuter-adjusted daytime population estimates" =    
#         total resident population + total workers working in area - total workers living in area

# For "Workers in Workplace Geography," see https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# "Total workers working in area:
# B08604 Total Workers for Workplace Geography
# B08604 is only available for data years 2011 and after. 
# The tables for workplace geography are only available for the following geographic summary levels: States; 
# Counties; Places; County Subdivisions in selected states (not MD); Combined Statistical Areas; Metropolitan 
# and Micropolitan Statistical Areas, and their associated Metropolitan Divisions and Principal Cities; 

census_data_workers = c.acs5.state_county(("NAME", 
                          "B08604_001E"),               
                          state_fips = "24",
                          county_fips = "510") 

# convert to dataframe
workers_df = pd.DataFrame(census_data_workers)
workers_df
                         

,NAME,B08604_001E,state,county
0,"Baltimore city, Maryland",383366.0,24,510


In [11]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract #
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
census_pd["GEOID"] = census_pd['state'] + census_pd['county'] + census_pd['tract']
census_pd.head()

,NAME,B19013_001E,B01003_001E,B01002_001E,B19301_001E,B17001_002E,B23025_005E,B23025_004E,B15003_017E,B15003_022E,...,B02001_008E,B03001_003E,B25008_002E,B28007_001E,B25003_002E,B25003_003E,state,county,tract,GEOID
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,0.0,0.0,306.0,1218.0,119.0,622.0,24,510,260403,24510260403
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,99.0,88.0,1030.0,1808.0,539.0,443.0,24,510,230200,24510230200
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,0.0,94.0,3674.0,3812.0,1384.0,323.0,24,510,260102,24510260102
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,29.0,66.0,164.0,716.0,88.0,368.0,24,510,260303,24510260303
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,10.0,970.0,751.0,1875.0,415.0,580.0,24,510,260800,24510260800


In [12]:
# number of rows = # of census tracts in the dataframe
print("Number of rows, columns: ", census_pd.shape)

Number of rows, columns:  (200, 22)


In [13]:
# Column Reordering
census_pd = census_pd.rename(columns={"B19013_001E": "Median household income",
                                      "B01003_001E": "Population",
                                      "B01002_001E": "Median age",
                                      "B19301_001E": "Per capita income", 
                                      "B17001_002E": "Poverty count",
                                      "B23025_004E": "# employed, age 16+",
                                      "B23025_005E": "Unemployment count",
                                      "B15003_017E": "# persons age 25+ graduated high school",
                                      "B15003_022E": "# persons age 25+ with Bachelor's degree",
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_008E": "Pop. 2 or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B25008_002E": "Total pop. in occupied housing units by tenure",
                                      "B25003_002E": "Total owner-occupied units",
                                      "B25003_003E": "Total renter-occupied units",
                                      "B28007_001E": "Labor force status by presence of computer/ internet in HH",
                                      "NAME": "Name", "state": "State", "GEOID": "GEOID"})

# Add in poverty rate (Poverty Count / Population)
census_pd["Poverty rate"] = 100 * \
    census_pd["Poverty count"].astype(
        int) / census_pd["Population"].astype(int)


# Add in unemployment rate (Employment Count / Population)
census_pd["Unemployment rate"] = 100 * \
    census_pd["Unemployment count"].astype(
        int) / census_pd["Population"].astype(int)
census_pd.head()

# Add in home ownership rate (# owner-occupied units / # of occupied housing units)

,Name,Median household income,Population,Median age,Per capita income,Poverty count,Unemployment count,"# employed, age 16+",# persons age 25+ graduated high school,# persons age 25+ with Bachelor's degree,...,Total pop. in occupied housing units by tenure,Labor force status by presence of computer/ internet in HH,Total owner-occupied units,Total renter-occupied units,State,county,tract,GEOID,Poverty rate,Unemployment rate
0,"Census Tract 2604.03, Baltimore city, Maryland",31625.0,1764.0,27.9,21744.0,436.0,34.0,885.0,450.0,91.0,...,306.0,1218.0,119.0,622.0,24,510,260403,24510260403,24.716553,1.927438
1,"Census Tract 2302, Baltimore city, Maryland",107391.0,1982.0,30.7,63973.0,151.0,17.0,1500.0,72.0,593.0,...,1030.0,1808.0,539.0,443.0,24,510,230200,24510230200,7.618567,0.857719
2,"Census Tract 2601.02, Baltimore city, Maryland",62586.0,4775.0,41.3,25903.0,469.0,151.0,2366.0,1231.0,321.0,...,3674.0,3812.0,1384.0,323.0,24,510,260102,24510260102,9.821990,3.162304
3,"Census Tract 2603.03, Baltimore city, Maryland",17000.0,969.0,37.0,19562.0,377.0,20.0,286.0,229.0,47.0,...,164.0,716.0,88.0,368.0,24,510,260303,24510260303,38.906089,2.063983
4,"Census Tract 2608, Baltimore city, Maryland",40795.0,2332.0,36.1,26895.0,481.0,38.0,1233.0,394.0,309.0,...,751.0,1875.0,415.0,580.0,24,510,260800,24510260800,20.626072,1.629503


In [15]:
# Final DataFrame
census_pd = census_pd[["State", "Name", "GEOID", "Population", "Median household income",
                       "Per capita income", "Poverty count", "Poverty rate", "Unemployment rate", 
                       "# employed, age 16+", "Unemployment count",
                      "# persons age 25+ graduated high school", "# persons age 25+ with Bachelor's degree",
                      "Median age","Pop. white", "Pop. Black", "Pop. 2 or more races", "Pop. Hispanic origin", 
                      "Total pop. in occupied housing units by tenure", "Total owner-occupied units", "Total renter-occupied units",
                      "Labor force status by presence of computer/ internet in HH"]]

census_pd.head()

,State,Name,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Labor force status by presence of computer/ internet in HH
0,24,"Census Tract 2604.03, Baltimore city, Maryland",24510260403,1764.0,31625.0,21744.0,436.0,24.716553,1.927438,885.0,...,91.0,27.9,23.0,1741.0,0.0,0.0,306.0,119.0,622.0,1218.0
1,24,"Census Tract 2302, Baltimore city, Maryland",24510230200,1982.0,107391.0,63973.0,151.0,7.618567,0.857719,1500.0,...,593.0,30.7,1683.0,169.0,99.0,88.0,1030.0,539.0,443.0,1808.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",24510260102,4775.0,62586.0,25903.0,469.0,9.821990,3.162304,2366.0,...,321.0,41.3,440.0,4254.0,0.0,94.0,3674.0,1384.0,323.0,3812.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",24510260303,969.0,17000.0,19562.0,377.0,38.906089,2.063983,286.0,...,47.0,37.0,281.0,626.0,29.0,66.0,164.0,88.0,368.0,716.0
4,24,"Census Tract 2608, Baltimore city, Maryland",24510260800,2332.0,40795.0,26895.0,481.0,20.626072,1.629503,1233.0,...,309.0,36.1,1748.0,310.0,10.0,970.0,751.0,415.0,580.0,1875.0


In [16]:
census_pd["Poverty rate"] = census_pd["Poverty rate"].astype(float).round(1)
census_pd.head()

C:\Users\Jenkir\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,State,Name,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Labor force status by presence of computer/ internet in HH
0,24,"Census Tract 2604.03, Baltimore city, Maryland",24510260403,1764.0,31625.0,21744.0,436.0,24.7,1.927438,885.0,...,91.0,27.9,23.0,1741.0,0.0,0.0,306.0,119.0,622.0,1218.0
1,24,"Census Tract 2302, Baltimore city, Maryland",24510230200,1982.0,107391.0,63973.0,151.0,7.6,0.857719,1500.0,...,593.0,30.7,1683.0,169.0,99.0,88.0,1030.0,539.0,443.0,1808.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",24510260102,4775.0,62586.0,25903.0,469.0,9.8,3.162304,2366.0,...,321.0,41.3,440.0,4254.0,0.0,94.0,3674.0,1384.0,323.0,3812.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",24510260303,969.0,17000.0,19562.0,377.0,38.9,2.063983,286.0,...,47.0,37.0,281.0,626.0,29.0,66.0,164.0,88.0,368.0,716.0
4,24,"Census Tract 2608, Baltimore city, Maryland",24510260800,2332.0,40795.0,26895.0,481.0,20.6,1.629503,1233.0,...,309.0,36.1,1748.0,310.0,10.0,970.0,751.0,415.0,580.0,1875.0


In [17]:
census_pd["Unemployment rate"] = census_pd["Unemployment rate"].astype(float).round(1)
census_pd.head()

C:\Users\Jenkir\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,State,Name,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,# persons age 25+ with Bachelor's degree,Median age,Pop. white,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Labor force status by presence of computer/ internet in HH
0,24,"Census Tract 2604.03, Baltimore city, Maryland",24510260403,1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,...,91.0,27.9,23.0,1741.0,0.0,0.0,306.0,119.0,622.0,1218.0
1,24,"Census Tract 2302, Baltimore city, Maryland",24510230200,1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,...,593.0,30.7,1683.0,169.0,99.0,88.0,1030.0,539.0,443.0,1808.0
2,24,"Census Tract 2601.02, Baltimore city, Maryland",24510260102,4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,...,321.0,41.3,440.0,4254.0,0.0,94.0,3674.0,1384.0,323.0,3812.0
3,24,"Census Tract 2603.03, Baltimore city, Maryland",24510260303,969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,...,47.0,37.0,281.0,626.0,29.0,66.0,164.0,88.0,368.0,716.0
4,24,"Census Tract 2608, Baltimore city, Maryland",24510260800,2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,...,309.0,36.1,1748.0,310.0,10.0,970.0,751.0,415.0,580.0,1875.0


In [18]:
census_pd.count()

State                                                         200
Name                                                          200
GEOID                                                         200
Population                                                    200
Median household income                                       200
Per capita income                                             200
Poverty count                                                 200
Poverty rate                                                  199
Unemployment rate                                             199
# employed, age 16+                                           200
Unemployment count                                            200
# persons age 25+ graduated high school                       200
# persons age 25+ with Bachelor's degree                      200
Median age                                                    200
Pop. white                                                    200
Pop. Black

In [19]:

census_pd[['Census_Tract', "County", "State Name"]]= census_pd['Name'].str.split(",", n=3, expand=True)
census_pd.head()

C:\Users\Jenkir\anaconda3\envs\PythonData\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,State,Name,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Labor force status by presence of computer/ internet in HH,Census_Tract,County,State Name
0,24,"Census Tract 2604.03, Baltimore city, Maryland",24510260403,1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,...,1741.0,0.0,0.0,306.0,119.0,622.0,1218.0,Census Tract 2604.03,Baltimore city,Maryland
1,24,"Census Tract 2302, Baltimore city, Maryland",24510230200,1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,...,169.0,99.0,88.0,1030.0,539.0,443.0,1808.0,Census Tract 2302,Baltimore city,Maryland
2,24,"Census Tract 2601.02, Baltimore city, Maryland",24510260102,4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,...,4254.0,0.0,94.0,3674.0,1384.0,323.0,3812.0,Census Tract 2601.02,Baltimore city,Maryland
3,24,"Census Tract 2603.03, Baltimore city, Maryland",24510260303,969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,...,626.0,29.0,66.0,164.0,88.0,368.0,716.0,Census Tract 2603.03,Baltimore city,Maryland
4,24,"Census Tract 2608, Baltimore city, Maryland",24510260800,2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,...,310.0,10.0,970.0,751.0,415.0,580.0,1875.0,Census Tract 2608,Baltimore city,Maryland


In [20]:
census_pd = census_pd.drop(["State"], axis=1)
census_pd.head()

,Name,GEOID,Population,Median household income,Per capita income,Poverty count,Poverty rate,Unemployment rate,"# employed, age 16+",Unemployment count,...,Pop. Black,Pop. 2 or more races,Pop. Hispanic origin,Total pop. in occupied housing units by tenure,Total owner-occupied units,Total renter-occupied units,Labor force status by presence of computer/ internet in HH,Census_Tract,County,State Name
0,"Census Tract 2604.03, Baltimore city, Maryland",24510260403,1764.0,31625.0,21744.0,436.0,24.7,1.9,885.0,34.0,...,1741.0,0.0,0.0,306.0,119.0,622.0,1218.0,Census Tract 2604.03,Baltimore city,Maryland
1,"Census Tract 2302, Baltimore city, Maryland",24510230200,1982.0,107391.0,63973.0,151.0,7.6,0.9,1500.0,17.0,...,169.0,99.0,88.0,1030.0,539.0,443.0,1808.0,Census Tract 2302,Baltimore city,Maryland
2,"Census Tract 2601.02, Baltimore city, Maryland",24510260102,4775.0,62586.0,25903.0,469.0,9.8,3.2,2366.0,151.0,...,4254.0,0.0,94.0,3674.0,1384.0,323.0,3812.0,Census Tract 2601.02,Baltimore city,Maryland
3,"Census Tract 2603.03, Baltimore city, Maryland",24510260303,969.0,17000.0,19562.0,377.0,38.9,2.1,286.0,20.0,...,626.0,29.0,66.0,164.0,88.0,368.0,716.0,Census Tract 2603.03,Baltimore city,Maryland
4,"Census Tract 2608, Baltimore city, Maryland",24510260800,2332.0,40795.0,26895.0,481.0,20.6,1.6,1233.0,38.0,...,310.0,10.0,970.0,751.0,415.0,580.0,1875.0,Census Tract 2608,Baltimore city,Maryland


In [ ]:
census_pd.columns

In [ ]:
# calculate daytime population; from: https://www.census.gov/topics/employment/commuting/guidance/calculations.html
# Two methods for calculating commuter-adjusted population estimates:

#Method 1: Total resident population + (total workers working in area - workers who lived and worked in same area) - (total workers living in area - workers who lived and worked in same area) OR;
#Method 2: Total resident population + Total workers working in area - Total workers living in area.




In [ ]:
census_pd["Census_Tract"] = census_pd['Census_Tract'].str.replace('Census Tract', "") 
census_pd

In [ ]:
census_pd = census_pd.drop(["State Name"], axis=1)
census_pd.head()

In [ ]:
# Calculate the number of unique census tracts in the DataFrame
tract_count = len(census_pd["Census_Tract"].unique())
tract_count

In [ ]:
census_final = census_pd.drop(["Name"], axis=1)
census_final.head()

In [ ]:
# Calculations can also be performed on Series and added into DataFrames as new columns
#thousands_of_dollars = data_file_df["Amount"]/1000
#data_file_df["Thousands of Dollars"] = thousands_of_dollars

#data_file_df.head()

In [ ]:
# Converting the membership days into weeks and then adding a column to the DataFrame
#weeks = training_df["Membership (Days)"]/7
#training_df["Membership (Weeks)"] = weeks

#training_df.head()

In [ ]:
# Export file as a CSV, without the Pandas index, but with the header
#census_pd.to_csv("Comm_Corridor_Stats_Draft2.csv", index = False, header=True)